In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
import warnings
import re

In [1]:
%%bash
source ~/.bashrc
conda activate ipyrad
bwa index ../repeatmasked/GCA_964030605.1_jaAcrPala1.1_genomic_shortened_headers.fa

ERROR: This cross-compiler package contains no program /storage/group/ibb3/default/.conda/envs/ipyrad/bin/x86_64-conda_cos6-linux-gnu-cc
ERROR: activate-gcc_linux-64.sh failed, see above for details


[bwa_index] Pack FASTA... 2.99 sec
[bwa_index] Construct BWT for the packed sequence...
[BWTIncCreate] textLength=671870000, availableWord=59275180
[BWTIncConstructFromPacked] 10 iterations done. 93848784 characters processed.
[BWTIncConstructFromPacked] 20 iterations done. 177149232 characters processed.
[BWTIncConstructFromPacked] 30 iterations done. 251182880 characters processed.
[BWTIncConstructFromPacked] 40 iterations done. 316980144 characters processed.
[BWTIncConstructFromPacked] 50 iterations done. 375456896 characters processed.
[BWTIncConstructFromPacked] 60 iterations done. 427427184 characters processed.
[BWTIncConstructFromPacked] 70 iterations done. 473614496 characters processed.
[BWTIncConstructFromPacked] 80 iterations done. 514661888 characters processed.
[BWTIncConstructFromPacked] 90 iterations done. 551140864 characters processed.
[BWTIncConstructFromPacked] 100 iterations done. 583559440 characters processed.
[BWTIncConstructFromPacked] 110 iterations done. 612

In [2]:
%%bash
source ~/.bashrc
conda activate ipyrad
mkdir -p ../linkage_map
cd ../linkage_map
bwa mem \
    ../repeatmasked/GCA_964030605.1_jaAcrPala1.1_genomic_shortened_headers.fa \
    ./probes.fasta \
    > ./probes_aln_apal.sam

ERROR: This cross-compiler package contains no program /storage/group/ibb3/default/.conda/envs/ipyrad/bin/x86_64-conda_cos6-linux-gnu-cc
ERROR: activate-gcc_linux-64.sh failed, see above for details


[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 53589 sequences (3803707 bp)...
[M::mem_process_seqs] Processed 53589 reads in 4.446 CPU sec, 4.873 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem ../repeatmasked/GCA_964030605.1_jaAcrPala1.1_genomic_shortened_headers.fa ./probes.fasta
[main] Real time: 6.253 sec; CPU: 4.802 sec


In [3]:
%%bash
source ~/.bashrc
conda activate ipyrad
cd ../linkage_map
sambamba view -t 4 --sam-input \
    -F "mapping_quality >= 3 and not (unmapped or secondary_alignment or supplementary or duplicate)" \
    ./probes_aln_apal.sam > ./unique_mappings_apal_probes.sam

ERROR: This cross-compiler package contains no program /storage/group/ibb3/default/.conda/envs/ipyrad/bin/x86_64-conda_cos6-linux-gnu-cc
ERROR: activate-gcc_linux-64.sh failed, see above for details



sambamba 0.7.1
 by Artem Tarasov and Pjotr Prins (C) 2012-2019
    LDC 1.20.0 / DMD v2.090.1 / LLVM7.0.0 / bootstrap LDC - the LLVM D compiler (0.17.6)



In [9]:
%%bash
cd ../linkage_map
cut -f1,3,4,5,6,10,12 unique_mappings_apal_probes.sam \
    > unique_mappings_apal_probes.tab

In [10]:
probes = pd.read_csv("../linkage_map/unique_mappings_apal_probes.tab", comment = "@", sep = "\t", names = ["probe", "contig", "start", "mapq", "cigar", "seq", "mismatch"])

In [11]:
probes = probes[probes["mapq"] > 20].reset_index(drop=True)
probes["npos"] = np.nan
for i in range(len(probes)):
    probes["npos"][i] = probes["seq"][i].find("N")
probes["new_snp_pos"] = np.nan
probes.loc[~probes["cigar"].str.contains("I|D"), "new_snp_pos"] = probes["npos"]

/tmp/ipykernel_352556/435527431.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  probes["npos"][i] = probes["seq"][i].find("N")


In [12]:
for i in range(len(probes)):
    cigar = re.findall('(\d+)?([A-Za-z])', probes["cigar"][i])
    length = len(cigar)
    if (length < 4) and (length > 2):
        if ( cigar[0][1] == "S" ) and ( cigar[1][1] == "M" ) and ( cigar[2][1] == "S" ):
            print
        elif ( cigar[0][1] == "M" ) and ( cigar[2][1] == "M" ):
            if int(cigar[0][0]) > probes["seq"][i].find("N"):
                probes["new_snp_pos"][i] = probes["seq"][i].find("N")
            elif int(cigar[0][0]) < probes["seq"][i].find("N"):
                if cigar[1][1] == "I":
                    if (int(cigar[0][0]) + int(cigar[1][0])) > probes["npos"][i]:
                        pass
                    else:
                        probes["new_snp_pos"][i] = probes["seq"][i].find("N") + int(cigar[1][0])
                elif cigar[1][1] == "D":
                    if (int(cigar[0][0]) + int(cigar[1][0])) > probes["npos"][i]:
                        pass
                    else:
                        probes["new_snp_pos"][i] = probes["seq"][i].find("N") - int(cigar[1][0])
    if (length > 3):
        if ( cigar[0][1] == "S" ) and ( cigar[1][1] == "M" ):
            if (int(cigar[0][0]) + int(cigar[1][0])) >= probes["seq"][i].find("N"):
                probes["new_snp_pos"][i] = probes["seq"][i].find("N")
    if ( cigar[0][1] == "M" ) and ( int(cigar[0][0]) >= probes["seq"][i].find("N") ):
        probes["new_snp_pos"][i] = probes["seq"][i].find("N")

/tmp/ipykernel_352556/1691801142.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  probes["new_snp_pos"][i] = probes["seq"][i].find("N")
/tmp/ipykernel_352556/1691801142.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  probes["new_snp_pos"][i] = probes["seq"][i].find("N")
/tmp/ipykernel_352556/1691801142.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  probes["new_snp_pos"][i] = probes["seq"][i].find("N") + int(cigar[1][0])
/

In [13]:
probes["new_snp_pos"] = probes["start"] + probes["new_snp_pos"]

In [14]:
probes = probes[probes["new_snp_pos"].notna()].reset_index(drop=True)
#now excluding probes w/ excessive mismatch due to indels --> probes.loc[probes["new_snp_pos"].isna(), "new_snp_pos"] = probes["start"] + probes["npos"]
probes["new_snp_pos"] = probes["new_snp_pos"].astype(int)

In [15]:
recommended = pd.read_csv("../linkage_map/recommended.txt", header = None)

In [16]:
probes = probes[probes["probe"].isin(recommended[0].tolist())].reset_index(drop=True)

In [17]:
file = open('../linkage_map/probes_in_apal.txt','w')
for item in probes["probe"].tolist():
    file.write(item+"\n")
file.close()

In [18]:
reformat = probes[["contig", "start", "probe"]]
reformat.columns = ["contig", "start", "ID"]

In [22]:
%%bash
cd ../linkage_map
grep -v "^##" ./Apalm_link_rename.vcf | cut -f1-3 > ./original_coords_apal.tab

In [23]:
tmp = pd.read_csv("../linkage_map/original_coords_apal.tab", sep = "\t")

In [24]:
tmp = tmp.merge(reformat, how = "left", on = "ID")

In [25]:
tmp["contig"] = tmp["contig"].fillna("NOMATCH")
tmp["start"] = tmp["start"].fillna(0)

In [26]:
reformatted_positions = tmp[["contig", "start", "ID"]]
reformatted_positions.columns = ["#CHROM", "POS", "ID"]

In [27]:
reformatted_positions["POS"] = reformatted_positions["POS"].astype(int)

/tmp/ipykernel_352556/2815286365.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reformatted_positions["POS"] = reformatted_positions["POS"].astype(int)


In [29]:
reformatted_positions.set_index("#CHROM").to_csv("../linkage_map/remapped_coords_apal.tab", sep = "\t")

In [30]:
%%bash
cd ../linkage_map/
grep -e "^##fileformat" -e "^##FORMAT" -e "^##INFO" \
    ./Apalm_link_rename.vcf \
    > ./Apalm_link_rename_remapped.vcf

In [31]:
%%bash
cd ../linkage_map
paste remapped_coords_apal.tab \
    <( grep -v "^##" Apalm_link_rename.vcf | cut -f4- ) \
    >> Apalm_link_rename_remapped.vcf

In [32]:
%%bash
cd ../linkage_map
grep "^#" Apalm_link_rename_remapped.vcf \
    > Apalm_link_rename_remapped_sorted.vcf
grep -v "^#" Apalm_link_rename_remapped.vcf \
    | grep -v "NOMATCH" \
    | sort -k1,1V -k2,2g \
    >> Apalm_link_rename_remapped_sorted.vcf

In [33]:
%%bash
source ~/.bashrc
conda activate ipyrad
cd ../linkage_map
vcftools --vcf Apalm_link_rename_remapped_sorted.vcf \
    --min-alleles 2 --max-alleles 2 --max-missing 0.8 --mac 3 \
    --recode --out apal_mote_samples_rehead_filtered_remapped_sorted

ERROR: This cross-compiler package contains no program /storage/group/ibb3/default/.conda/envs/ipyrad/bin/x86_64-conda_cos6-linux-gnu-cc
ERROR: activate-gcc_linux-64.sh failed, see above for details



VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf Apalm_link_rename_remapped_sorted.vcf
	--mac 3
	--max-alleles 2
	--min-alleles 2
	--max-missing 0.8
	--out apal_mote_samples_rehead_filtered_remapped_sorted
	--recode

After filtering, kept 107 out of 107 Individuals
Outputting VCF file...
After filtering, kept 4968 out of a possible 24294 Sites
Run Time = 3.00 seconds


In [34]:
!mkdir -p ../linkage_map/lepwrap_apal

In [ ]:
LepWrap 40 config_round1.yml

In [1]:
%%bash
cd ../linkage_map/lepwrap_apal/3_SeparateChromosomes/
cat ./all.LOD.summary | tr -s " " > tmp.txt

In [4]:
lg = pd.read_csv("../linkage_map/lepwrap_apal/3_SeparateChromosomes/tmp.txt", sep = " ")

In [6]:
lg[lg.columns[4:]].head(50)

,LOD.4,LOD.5,LOD.6,LOD.7,LOD.8,LOD.9,LOD.10,LOD.11,LOD.12,LOD.13,LOD.14,LOD.15,LOD.16,LOD.17,LOD.18,LOD.19,LOD.20
0,1146,1146,1146,1146,1146,1146,1146,1148,1149,1161,1166,1196,1223,1282,1300,1318,1349
1,510,403,403,403,403,403,403,403,396,394,394,374,328,116,99,78,78
2,474,274,274,274,274,274,274,272,272,272,264,263,191,115,84,68,68
3,403,249,249,249,249,249,249,249,249,244,244,229,128,105,78,63,56
4,397,245,245,245,245,245,245,245,245,233,233,209,110,104,78,59,55
5,245,236,236,236,236,236,236,236,236,228,228,207,109,95,72,56,51
6,196,225,225,225,225,225,225,225,225,225,225,198,99,88,70,55,50
7,189,209,209,209,209,209,209,209,209,209,209,191,95,78,69,53,48
8,175,196,196,196,196,196,196,196,196,195,191,178,93,78,63,51,47
9,164,189,189,189,189,189,189,189,189,188,188,172,88,76,59,48,45
